## Example of GCA

### Node-level representation learning on Amazon Photos

In [ ]:
from dig.sslgraph.utils import Encoder
from dig.sslgraph.evaluation import NodeUnsupervised
from dig.sslgraph.dataset import get_node_dataset
from dig.sslgraph.method import pGRACE
from dig.sslgraph.utils.adaptive import generate_split

#### Loading dataset

In [ ]:
from torch_geometric.datasets import Planetoid, CitationFull, WikiCS, Coauthor, Amazon
import torch_geometric.transforms as T
import torch
path = './dig-experiments/'
dataset = Amazon(root=path, name='Photo', transform=T.NormalizeFeatures())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
data = dataset[0].to(device=device)

In [ ]:
split = generate_split(data.num_nodes, train_ratio = 0.1, val_ratio = 0.1)

#### Define encoders and contrastive model
##### You can refer to [https://github.com/CRIPAC-DIG/GCA/tree/main/param](https://github.com/CRIPAC-DIG/GCA/tree/main/param) for detailed training configs.
##### ***Note***: Results in the GCA paper uses different training-test splits to the public splits, due to which you may see slightly different results in DIG and the original implementation of GCA.

In [ ]:
embed_dim = 256
encoder = Encoder(feat_dim=dataset[0].x.shape[1], hidden_dim=embed_dim, 
                  n_layers=2, gnn='gcn', act='relu', node_level=True, graph_level=False).to(device)
grace = pGRACE(dim=embed_dim, proj_n_dim=64, centrality_measure='degree', prob_edge_1 = 0.3, prob_edge_2 = 0.5, prob_feature_1 = 0.1, prob_feature_2 = 0.1, p_tau=0.7, tau=0.1, device=device)

In [ ]:
evaluator = NodeUnsupervised(dataset, train_mask=split[0], test_mask = split[1], val_mask = split[2], device=device, log_interval=100)
evaluator.setup_train_config(p_lr=0.1, p_epoch=2000, p_weight_decay=1e-5, comp_embed_on='cuda:0')
evaluator.evaluate(learning_model=grace, encoder=encoder)